Download all dependencies

In [9]:
%pip install stanza -U
%pip install accelerate -U
%pip install -U pip setuptools wheel textacy

Import Libraries

In [1]:
import Linguistic_Metrics as lm
import json
import timeit
import string
import nltk
import spacy
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Error loading stanford-corenlp: Package 'stanford-corenlp'
[nltk_data]     not found in index
[nltk_data] Error loading stanford-corenlp-4.2.0: Package 'stanford-
[nltk_data]     corenlp-4.2.0' not found in index


2024-02-21 14:42:50 INFO: Downloading default packages for language: en (English) ...
2024-02-21 14:42:51 INFO: File exists: C:\Users\aryam\stanza_resources\en\default.zip
2024-02-21 14:42:55 INFO: Finished downloading models and saved to C:\Users\aryam\stanza_resources.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\aryam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aryam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Helper Functions

In [13]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

def replace_punctuation_with_whitespace(input_string):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    return input_string.translate(translator)

Initialize class instances 

In [14]:
%cd D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data
data = json.load(open(r'pairwise_evaluation_w_embeddings.json'))
data_nat = json.load(open(r'final_metrics_naturalness.json'))
df = pd.DataFrame(data_nat)
nat = lm.naturalness(df)
model = lm.LinguisticFeatures()

D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data


2024-02-21 15:13:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-02-21 15:13:59 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus                   |
| ner          | ontonotes-ww-multi_charlm |

2024-02-21 15:13:59 INFO: Using device: cpu
2024-02-21 15:13:59 INFO: Loading: tokenize
2024-02-21 15:13:59 INFO: Loading: mwt
2024-02-21 15:13:59 INFO: Loading: pos
2024-02-21 15:14:00 INFO: Loading: lemma
2024-02-21 15:14:00 INFO: Loading: constituency
2024-02-21 15:14:01 INFO: Loading: depparse
2024-02-21 15:14:01 INFO: Loading: sentiment
2024-02-21 15:14:01 INFO: Loading: ner
2024-02-21 15:14:02 INFO: Done loading processors!


Example for calculating naturalness for any text 

Problem: If we use very few words in a sentence and use more number of sentences, the score exceeds 1 sometimes

Potential Reason: The range for min max scaling is based on the dataset, for custom text the min max might change leading to score above 1 or less than 0.

In [15]:
score = nat.calculate_naturalness('Evans said officers negotiated with the suspect while additional officers responded to the scene.During those negotiations, the man opened fired, killing Elmstrand, Ruge and Finseth, and injuring Medlicott.')
score

0.7154226525399803

System Analysis

Generate all UPOS tags for the dataset

In [10]:
tags_writer,tags_llm,tags_article = model.spacy_tagger_dataset(data)

save_json(tags_writer,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_writer.json')
save_json(tags_llm,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_llm.json')
save_json(tags_article,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_article.json')

KeyboardInterrupt: 

In [17]:
tags_writer = json.load(open(r'tags_writer.json'))
tags_llm= json.load(open(r'tags_llm.json'))
tags_article= json.load(open(r'tags_article.json'))

Calculate percentage of tags used 

In [18]:
tags_per_writer = model.count_tags_percentage(tags_writer)
tags_per_llm = model.count_tags_percentage(tags_llm)
tags_per_article = model.count_tags_percentage(tags_article)

print(tags_per_writer,tags_per_llm,tags_per_article)

save_json(tags_per_writer,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_percentage_writer.json')
save_json(tags_per_llm,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_percentage_llm.json')
save_json(tags_per_article,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/tags_percentage_article.json')

{'DET': 9.623379799461972, 'NOUN': 20.78136463683052, 'ADP': 11.65321594521888, 'PROPN': 9.280997798972853, 'VERB': 11.335289801907557, 'CCONJ': 2.3691611640988017, 'PRON': 4.01076057715823, 'PUNCT': 9.403277084861825, 'AUX': 5.5392516507703595, 'ADJ': 6.807899241868427, 'SCONJ': 1.7057960381511372, 'SPACE': 0.9384935191978478, 'NUM': 1.6966250917094645, 'ADV': 2.528124235754463, 'PART': 2.3080215211543167, 'X': 0.018341892883345562} {'DET': 10.157815631262524, 'NOUN': 20.052605210420843, 'ADP': 11.062750501002004, 'PROPN': 12.033441883767534, 'AUX': 5.573647294589178, 'VERB': 11.137900801603207, 'PART': 2.6928857715430863, 'PUNCT': 9.772670340681362, 'PRON': 3.547720440881764, 'ADJ': 5.92121743486974, 'ADV': 1.8881513026052104, 'CCONJ': 2.3327905811623246, 'NUM': 1.7566382765531061, 'SCONJ': 1.8474448897795592, 'SYM': 0.2035320641282565, 'X': 0.018787575150300603} {'PROPN': 9.368373536250905, 'PART': 2.6149282492669403, 'NOUN': 17.220934599292164, 'AUX': 4.8357300822013425, 'VERB': 10

Get all writer and LLM summaries

In [19]:
writer_summs = []
llm_summs = []
article_summs = []
rem_list = ['full_embedding', 'writer_summary_embedding', 'llm_summary_embedding']
for key in rem_list:
    for datum in data:
        writer_summs.append(datum['writer_summary'])
        llm_summs.append(datum['text-davinci-002_summary'])
        article_summs.append(datum['article_text'])
        del datum[key]

Calculate average dependency Tree heights for the whole dataset.(run the cell above before running this)

In [12]:
tree_ht_article = model.calculate_average_tree_height(article_summs)
tree_ht_writer = model.calculate_average_tree_height(writer_summs)
tree_ht_llm = model.calculate_average_tree_height(llm_summs)
print(tree_ht_writer,tree_ht_llm,tree_ht_article)

3.8430239667400343 4.02686623246493 4.0256846002404965


Count dependency arcs based metrics

In [20]:
arcs_writer = model.count_dependency_arcs(writer_summs)
arcs_llm = model.count_dependency_arcs(llm_summs)
arcs_article = model.count_dependency_arcs(article_summs)

print(arcs_writer,arcs_llm,arcs_article)

# save_json(arcs_writer,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/arcs1_writer.json')
# save_json(arcs_llm,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/arcs_llm1.json')
# save_json(arcs_article,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/arcs_article1.json')

{'number of sentences': 4788, 'average left arc length': 1.9764058198977585, 'average right arc length': 3.7244923697818137, 'average total arc length': 2.8673029952436044, 'right arc percentage': 50.964134207481685, 'left arc percentage': 49.035865792518315, 'standard deviation left arcs': 2.1279911149463357, 'standard deviation right arcs': 4.595775991690084, 'standard deviation total arcs': 3.7078829718281128} {'number of sentences': 4146, 'average left arc length': 1.8653553215444731, 'average right arc length': 4.3774183112261404, 'average total arc length': 3.1360214701839366, 'right arc percentage': 50.58257511291484, 'left arc percentage': 49.41742488708516, 'standard deviation left arcs': 1.7973213647715085, 'standard deviation right arcs': 5.41783435367229, 'standard deviation total arcs': 4.2451420578712495} {'number of sentences': 67140, 'average left arc length': 2.1942593671623642, 'average right arc length': 4.147497985255052, 'average total arc length': 3.23120575523608

Calculate average subtree analysis for the whole dataset.

In [22]:
word_avg_llm = model.calculate_subtree_features(llm_summs)
print(word_avg_llm)
word_avg_article = model.calculate_subtree_features(article_summs)
print(word_avg_article)
word_avg_writer = model.calculate_subtree_features(writer_summs)
print(word_avg_writer)

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>

Calculate average sentence length for all summaries in the dataset

In [21]:
average_length_writer,average_length_llm,average_length_article = model.avg_sentence_length_dataset(data)
print(average_length_writer,average_length_llm,average_length_article)

18.154671717171716 20.263386396526773 19.716268486916952


Calculate average word lengths for all summaries in the dataset

In [22]:
word_avg_llm = model.average_word_length_system(llm_summs)
print(word_avg_llm)
word_avg_article = model.average_word_length_system(article_summs)
print(word_avg_article)
word_avg_writer = model.average_word_length_system(writer_summs)
print(word_avg_writer)

4.858397518558533
4.599166289651323
4.809794325998426


Calculate Gender Bias in the dataset

In [ ]:
writer_ratio, llm_ratio, article_ratio = model.gender_ratio_dataset(data)
print(writer_ratio, llm_ratio, article_ratio)

Generate all analysis metrics for single data points

Calculating all the linguistic features for each data point in the dataset and creating a new document for analysis.

In [ ]:
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data)))
    avg_len_writer = model.average_word_length_system([datum['writer_summary']])
    avg_len_llm = model.average_word_length_system([datum['text-davinci-002_summary']])
    avg_len_article = model.average_word_length_system([datum['article_text']])
    tree_ht_article = model.calculate_average_tree_height([datum['article_text']])
    tree_ht_llm = model.calculate_average_tree_height([datum['text-davinci-002_summary']])
    tree_ht_writer = model.calculate_average_tree_height([datum['writer_summary']])
    tags_writer = model.spacy_tagger(datum['writer_summary'])
    tags_llm = model.spacy_tagger(datum['text-davinci-002_summary'])
    tags_article = model.spacy_tagger(datum['article_text'])
    tags_per_writer = model.count_tags_percentage([tags_writer])
    tags_per_llm = model.count_tags_percentage([tags_llm])
    tags_per_article = model.count_tags_percentage([tags_article])
    sen_len_writer = model.avg_sentence_length(datum['writer_summary'])
    sen_len_llm = model.avg_sentence_length(datum['text-davinci-002_summary'])
    sen_len_article = model.avg_sentence_length(datum['article_text'])
    arcs_writer = model.count_dependency_arcs([datum['writer_summary']])
    arcs_llm = model.count_dependency_arcs([datum['text-davinci-002_summary']])
    arcs_article = model.count_dependency_arcs([datum['article_text']])
    subtree_writer = model.calculate_subtree_features(datum['writer_summary'])
    subtree_llm = model.calculate_subtree_features(datum['text-davinci-002_summary'])
    subtree_article = model.calculate_subtree_features(datum['article_text'])
    gender_writer = model.gender_ratio_single(datum['writer_summary'])
    gender_llm = model.gender_ratio_single(datum['text-davinci-002_summary'])
    gender_article = model.gender_ratio_single(datum['article_text'])

    datum['average word lengths'] = {"writer":avg_len_writer,"LLM":avg_len_llm,"article":avg_len_article}
    datum['Average Dependency tree heights'] = {"writer":tree_ht_writer,"LLM":tree_ht_llm,"article":tree_ht_article}
    datum['Percentage of UPOS tags used'] = {"writer":tags_per_writer,"LLM":tags_per_llm,"article":tags_per_article}
    datum['average sentence lengths'] = {"writer":sen_len_writer,"LLM":sen_len_llm,"article":sen_len_article}
    datum['Arc analysis'] = {"writer":arcs_writer,"LLM":arcs_llm,"article":arcs_article}
    datum['Gender Ratios'] = {"writer":gender_writer,"LLM":gender_llm,"article":gender_article}
    datum['Left and right subtrees analysis'] = {"writer":model.calculate_subtree_features(datum['writer_summary']),"LLM":model.calculate_subtree_features(datum['text-davinci-002_summary']),"article":model.calculate_subtree_features(datum['article_text'])}


save_json(data,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/data_analysis.json')

0/599
1/599
2/599
3/599
4/599
5/599
6/599
7/599
8/599
9/599
10/599
11/599
12/599
13/599
14/599
15/599
16/599
17/599
18/599
19/599
20/599
21/599
22/599
23/599
24/599
25/599
26/599
27/599
28/599
29/599
30/599
31/599
32/599
33/599
34/599
35/599
36/599
37/599
38/599
39/599
40/599
41/599
42/599
43/599
44/599
45/599
46/599
47/599
48/599
49/599
50/599
51/599
52/599
53/599
54/599
55/599
56/599
57/599
58/599
59/599
60/599
61/599
62/599
63/599
64/599
65/599
66/599
67/599
68/599
69/599
70/599
71/599
72/599
73/599
74/599
75/599
76/599
77/599
78/599
79/599
80/599
81/599
82/599
83/599
84/599
85/599
86/599
87/599
88/599
89/599
90/599
91/599
92/599
93/599
94/599
95/599
96/599
97/599
98/599
99/599
100/599
101/599
102/599
103/599
104/599
105/599
106/599
107/599
108/599
109/599
110/599
111/599
112/599
113/599
114/599
115/599
116/599
117/599
118/599
119/599
120/599
121/599
122/599
123/599
124/599
125/599
126/599
127/599
128/599
129/599
130/599
131/599
132/599
133/599
134/599
135/599
136/599
137/599
138/59

In [7]:
analysis_data = json.load(open(r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/data_analysis.json'))

Create Delta values document for calculating naturalness(Not used in actual calculation, but used in analysis of metrics to be selected)

In [ ]:
list_articles = []
for i,datum in enumerate(analysis_data):
    print('{}/{}'.format(i, len(data)))
    natural_dict = {}
    natural_dict['article_id'] = datum['article_id']
    natural_dict['writer_summary'] = datum['writer_summary']
    natural_dict['text-davinci-002_summary'] = datum['text-davinci-002_summary']
    natural_dict['overall_writer_better'] = datum['overall_writer_better']
    natural_dict['informative_writer_better'] = datum['informative_writer_better']
    natural_dict['delta average word lengths'] = datum['average word lengths']['writer'] - datum['average word lengths']['LLM']
    natural_dict['delta Average Dependency tree heights'] = datum['Average Dependency tree heights']['writer'] - datum['Average Dependency tree heights']['LLM']
    natural_dict['delta_average_sentence_lengths'] = datum['average sentence lengths']['writer'] - datum['average sentence lengths']['LLM']
    natural_dict['delta_average_left_arc_length'] = datum['Arc analysis']['writer']['average left arc length'] - datum['Arc analysis']['LLM']['average left arc length']
    natural_dict['delta_average_right_arc_length'] = datum['Arc analysis']['writer']['average right arc length'] - datum['Arc analysis']['LLM']['average right arc length']
    natural_dict['delta_average_right_arc_length'] = datum['Arc analysis']['writer']['average total arc length'] - datum['Arc analysis']['LLM']['average total arc length']
    natural_dict['delta_avg_left_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_left_subtree_height'] - datum['Left and right subtrees analysis']['LLM']['avg_left_subtree_height']
    natural_dict['delta_avg_right_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_right_subtree_height'] - datum['Left and right subtrees analysis']['LLM']['avg_right_subtree_height']
    natural_dict['delta_num_left_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_left_subtrees'] - datum['Left and right subtrees analysis']['LLM']['num_left_subtrees']
    natural_dict['delta_num_right_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_right_subtrees'] - datum['Left and right subtrees analysis']['LLM']['num_right_subtrees']
    natural_dict['delta_no_sentences'] = datum['Arc analysis']['writer']['number of sentences'] - datum['Arc analysis']['LLM']['number of sentences']
    natural_dict['no_sentences_writer'] = datum['Arc analysis']['writer']['number of sentences']
    natural_dict['no_sentences_llm'] = datum['Arc analysis']['LLM']['number of sentences']
    list_articles.append(natural_dict)
save_json(list_articles,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/delta_metrics_natural.json')

0/599
1/599
2/599
3/599
4/599
5/599
6/599
7/599
8/599
9/599
10/599
11/599
12/599
13/599
14/599
15/599
16/599
17/599
18/599
19/599
20/599
21/599
22/599
23/599
24/599
25/599
26/599
27/599
28/599
29/599
30/599
31/599
32/599
33/599
34/599
35/599
36/599
37/599
38/599
39/599
40/599
41/599
42/599
43/599
44/599
45/599
46/599
47/599
48/599
49/599
50/599
51/599
52/599
53/599
54/599
55/599
56/599
57/599
58/599
59/599
60/599
61/599
62/599
63/599
64/599
65/599
66/599
67/599
68/599
69/599
70/599
71/599
72/599
73/599
74/599
75/599
76/599
77/599
78/599
79/599
80/599
81/599
82/599
83/599
84/599
85/599
86/599
87/599
88/599
89/599
90/599
91/599
92/599
93/599
94/599
95/599
96/599
97/599
98/599
99/599
100/599
101/599
102/599
103/599
104/599
105/599
106/599
107/599
108/599
109/599
110/599
111/599
112/599
113/599
114/599
115/599
116/599
117/599
118/599
119/599
120/599
121/599
122/599
123/599
124/599
125/599
126/599
127/599
128/599
129/599
130/599
131/599
132/599
133/599
134/599
135/599
136/599
137/599
138/59

Create document for the final metrics used for calculating naturalness

In [8]:
list_articles = []
for i,datum in enumerate(analysis_data):
    print('{}/{}'.format(i, len(data)))
    natural_dict = {}
    natural_dict['article_id'] = datum['article_id']
    natural_dict['writer_summary'] = datum['writer_summary']
    natural_dict['text-davinci-002_summary'] = datum['text-davinci-002_summary']
    natural_dict['overall_writer_better'] = datum['overall_writer_better']
    natural_dict['informative_writer_better'] = datum['informative_writer_better']
    natural_dict['writer average word lengths'] = datum['average word lengths']['writer'] 
    natural_dict['LLM average word lengths'] = datum['average word lengths']['LLM']
    natural_dict['Writer Average Dependency tree heights'] = datum['Average Dependency tree heights']['writer']
    natural_dict['LLM Average Dependency tree heights'] = datum['Average Dependency tree heights']['LLM']
    natural_dict['writer_average_sentence_lengths'] = datum['average sentence lengths']['writer']
    natural_dict['LLM_average_sentence_lengths'] = datum['average sentence lengths']['LLM']
    natural_dict['writer_average_left_arc_length'] = datum['Arc analysis']['writer']['average left arc length']
    natural_dict['LLM_average_left_arc_length'] = datum['Arc analysis']['LLM']['average left arc length']
    natural_dict['writer_average_right_arc_length'] = datum['Arc analysis']['writer']['average right arc length']
    natural_dict['LLM_average_right_arc_length'] = datum['Arc analysis']['LLM']['average right arc length']
    natural_dict['writer_average_total_arc_length'] = datum['Arc analysis']['writer']['average total arc length']
    natural_dict['LLM_average_total_arc_length'] = datum['Arc analysis']['LLM']['average total arc length']
    natural_dict['writer_avg_left_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_left_subtree_height'] 
    natural_dict['LLM_avg_left_subtree_height'] = datum['Left and right subtrees analysis']['LLM']['avg_left_subtree_height']
    natural_dict['writer_avg_right_subtree_height'] = datum['Left and right subtrees analysis']['writer']['avg_right_subtree_height']
    natural_dict['LLM_avg_right_subtree_height'] = datum['Left and right subtrees analysis']['LLM']['avg_right_subtree_height']
    natural_dict['writer_num_left_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_left_subtrees']
    natural_dict['LLM_num_left_subtrees'] = datum['Left and right subtrees analysis']['LLM']['num_left_subtrees']
    natural_dict['writer_num_right_subtrees'] = datum['Left and right subtrees analysis']['writer']['num_right_subtrees']
    natural_dict['LLM_num_right_subtrees'] = datum['Left and right subtrees analysis']['LLM']['num_right_subtrees']
    # natural_dict['delta_no_sentences'] = datum['Arc analysis']['writer']['number of sentences'] - datum['Arc analysis']['LLM']['number of sentences']
    # natural_dict['no_sentences_writer'] = datum['Arc analysis']['writer']['number of sentences']
    # natural_dict['no_sentences_llm'] = datum['Arc analysis']['LLM']['number of sentences']
    list_articles.append(natural_dict)

0/599
1/599
2/599
3/599
4/599
5/599
6/599
7/599
8/599
9/599
10/599
11/599
12/599
13/599
14/599
15/599
16/599
17/599
18/599
19/599
20/599
21/599
22/599
23/599
24/599
25/599
26/599
27/599
28/599
29/599
30/599
31/599
32/599
33/599
34/599
35/599
36/599
37/599
38/599
39/599
40/599
41/599
42/599
43/599
44/599
45/599
46/599
47/599
48/599
49/599
50/599
51/599
52/599
53/599
54/599
55/599
56/599
57/599
58/599
59/599
60/599
61/599
62/599
63/599
64/599
65/599
66/599
67/599
68/599
69/599
70/599
71/599
72/599
73/599
74/599
75/599
76/599
77/599
78/599
79/599
80/599
81/599
82/599
83/599
84/599
85/599
86/599
87/599
88/599
89/599
90/599
91/599
92/599
93/599
94/599
95/599
96/599
97/599
98/599
99/599
100/599
101/599
102/599
103/599
104/599
105/599
106/599
107/599
108/599
109/599
110/599
111/599
112/599
113/599
114/599
115/599
116/599
117/599
118/599
119/599
120/599
121/599
122/599
123/599
124/599
125/599
126/599
127/599
128/599
129/599
130/599
131/599
132/599
133/599
134/599
135/599
136/599
137/599
138/59

Define range(range is min in both LLM and Writer summaries same for max to account for texts not in dataset) for selected features and min max scaling for selected features. These are the final values used for naturalness

In [ ]:
for dat in list_articles:
    range_avg_dep_tree_ht = (min(df['Writer Average Dependency tree heights'].min(),df['LLM Average Dependency tree heights'].min()),max(df['Writer Average Dependency tree heights'].max(),df['LLM Average Dependency tree heights'].max()))
    range_avg_sentence_length = (min(df['writer_average_sentence_lengths'].min(),df['LLM_average_sentence_lengths'].min()),max(df['writer_average_sentence_lengths'].max(),df['LLM_average_sentence_lengths'].max()))
    range_avg_left_subtree_ht = (min(df['writer_avg_left_subtree_height'].min(),df['LLM_avg_left_subtree_height'].min()),max(df['writer_avg_left_subtree_height'].max(),df['LLM_avg_left_subtree_height'].max()))
    range_avg_right_subtree_ht = (min(df['writer_avg_right_subtree_height'].min(),df['LLM_avg_right_subtree_height'].min()),max(df['writer_avg_right_subtree_height'].max(),df['LLM_avg_right_subtree_height'].max()))
    dat['Writer Average Dependency tree heights'] = (dat['Writer Average Dependency tree heights']-range_avg_dep_tree_ht[0])/(range_avg_dep_tree_ht[1]-range_avg_dep_tree_ht[0])
    dat['LLM Average Dependency tree heights'] = (dat['LLM Average Dependency tree heights']-range_avg_dep_tree_ht[0])/(range_avg_dep_tree_ht[1]-range_avg_dep_tree_ht[0])
    dat['writer_average_sentence_lengths'] = (dat['writer_average_sentence_lengths']-range_avg_sentence_length[0])/(range_avg_sentence_length[1]-range_avg_sentence_length[0])
    dat['LLM_average_sentence_lengths'] = (dat['LLM_average_sentence_lengths']-range_avg_sentence_length[0])/(range_avg_sentence_length[1]-range_avg_sentence_length[0])
    dat['writer_avg_left_subtree_height'] = (dat['writer_avg_left_subtree_height']-range_avg_left_subtree_ht[0])/(range_avg_left_subtree_ht[1]-range_avg_left_subtree_ht[0])
    dat['LLM_avg_left_subtree_height'] = (dat['LLM_avg_left_subtree_height']-range_avg_left_subtree_ht[0])/(range_avg_left_subtree_ht[1]-range_avg_left_subtree_ht[0])
    dat['writer_avg_right_subtree_height'] = (dat['writer_avg_right_subtree_height']-range_avg_right_subtree_ht[0])/(range_avg_right_subtree_ht[1]-range_avg_right_subtree_ht[0])
    dat['LLM_avg_right_subtree_height'] = (dat['LLM_avg_right_subtree_height']-range_avg_right_subtree_ht[0])/(range_avg_right_subtree_ht[1]-range_avg_right_subtree_ht[0])
save_json(list_articles,r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/final_metrics_naturalness.json')

Dataframe for final features

In [15]:
natural_metrics_dict = json.load(open(r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/final_metrics_naturalness.json'))
df_features = pd.DataFrame(natural_metrics_dict)
df_features.columns

Load delta values document for feature importance measurement (not used directly for naturalness)

In [7]:
data_natural =  json.load(open(r'D:/Laptop_Projects/LLMEval-1/preprocess/experiments/data/metrics_natural.json'))
df_delta = pd.DataFrame(data_natural)

Random forest classifier training to assign weights to the features

In [ ]:
feature_writer = [
    'writer average word lengths',
    # 'LLM average word lengths',
    'Writer Average Dependency tree heights',
    # 'LLM Average Dependency tree heights',
    'writer_average_sentence_lengths', 
    # 'LLM_average_sentence_lengths',
    # 'writer_average_left_arc_length', 
    # 'LLM_average_left_arc_length',
    # 'writer_average_right_arc_length', 
    # 'LLM_average_right_arc_length',
    # 'writer_average_total_arc_length', 
    # 'LLM_average_total_arc_length',
    'writer_avg_left_subtree_height',
    # 'LLM_avg_left_subtree_height',
    'writer_avg_right_subtree_height', 
    # 'LLM_avg_right_subtree_height',
    # 'writer_num_left_subtrees', 
    # 'LLM_num_left_subtrees',
    # 'writer_num_right_subtrees', 
    # 'LLM_num_right_subtrees'
]
feature_LLM = [
    # 'writer average word lengths',
    'LLM average word lengths',
    # 'Writer Average Dependency tree heights',
    'LLM Average Dependency tree heights',
    # 'writer_average_sentence_lengths', 
    'LLM_average_sentence_lengths',
    # 'writer_average_left_arc_length', 
    # 'LLM_average_left_arc_length',
    # 'writer_average_right_arc_length', 
    # 'LLM_average_right_arc_length',
    # 'writer_average_total_arc_length', 
    # 'LLM_average_total_arc_length',
    # 'writer_avg_left_subtree_height',
    'LLM_avg_left_subtree_height',
    # 'writer_avg_right_subtree_height', 
    'LLM_avg_right_subtree_height',
    # 'writer_num_left_subtrees', 
    # 'LLM_num_left_subtrees',
    # 'writer_num_right_subtrees', 
    # 'LLM_num_right_subtrees'
]

# encode target variable
encoder = LabelEncoder()
df['overall_writer_better'] = [str(value) for value in df['overall_writer_better']]
df['informative_writer_better'] = [str(value) for value in df['informative_writer_better']]
df['overall_writer_better'] = encoder.fit_transform(df['overall_writer_better'])
df['informative_writer_better'] = encoder.fit_transform(df['informative_writer_better'])

X_train, X_test, y_train, y_test = train_test_split(df[feature_writer], df['overall_writer_better'], test_size=0.2, random_state=42) # change to feature_LLM for LLM weights

# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

importances = clf.feature_importances_
for feature, importance in zip(feature_writer, importances):
    print(f"{feature}: {importance}")


writer average word lengths: 0.22546514183292254
Writer Average Dependency tree heights: 0.21132551661766943
writer_average_sentence_lengths: 0.18634917042317664
writer_avg_left_subtree_height: 0.1741745467970967
writer_avg_right_subtree_height: 0.20268562432913467


Calculating naturalness for all writer and LLM summaries in the dataset(Same weights are taken for both to prevent bias in one category)

In [26]:

weights_writer = {
    # 'writer average word lengths': 0.29729001098302515,
    'Writer Average Dependency tree heights': 0.2826426317853948,
    'writer_average_sentence_lengths': 0.2522139996530825,
    # 'writer_average_left_arc_length': 0.09813570839334515,
    # 'writer_average_right_arc_length': 0.10403547536278904,
    # 'writer_average_total_arc_length': 0.10209866577981681,
    'writer_avg_left_subtree_height': 0.23287886122532872,
    'writer_avg_right_subtree_height': 0.2458490930520618,
    # 'writer_num_left_subtrees': 0.2614301716393748,
    # 'writer_num_right_subtrees': 0.23331946113659002
}

weights_llm = {
    # 'LLM average word lengths': 0.24348774347757085,
    'LLM Average Dependency tree heights': 0.2826426317853948,
    'LLM_average_sentence_lengths': 0.2522139996530825,
    # 'LLM_average_left_arc_length': 0.08065295776913092,
    # 'LLM_average_right_arc_length': 0.08178144517768533,
    # 'LLM_average_total_arc_length': 0.07702517940148237,
    'LLM_avg_left_subtree_height': 0.23287886122532872,
    'LLM_avg_right_subtree_height': 0.2458490930520618,
    # 'LLM_num_left_subtrees': 0.2545225325226861,
    # 'LLM_num_right_subtrees': 0.27698367937873053
}

def calculate_naturalness_score(features, weights):
    inverses = 1.0-features
    naturalness_score = np.mean(np.dot(inverses, weights))
    return naturalness_score

df_features['naturalness_score_llm'] = df_features.apply(lambda row: calculate_naturalness_score(row[weights_llm.keys()], np.array(list(weights_llm.values()))), axis=1)
df_features['naturalness_score_writer'] = df_features.apply(lambda row: calculate_naturalness_score(row[weights_writer.keys()], np.array(list(weights_writer.values()))), axis=1)

In [32]:
csv = df_features.to_csv('df_features.csv', index = True) 

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

feature_cols = [
    'delta average word lengths', 
    'delta Average Dependency tree heights',
    'delta_average_sentence_lengths',
    'delta_average_left_arc_length',
    'delta_average_right_arc_length',
    'delta_avg_left_subtree_height',
    'delta_avg_right_subtree_height',
    'delta_num_left_subtrees',
    'delta_num_right_subtrees',
    'delta_no_sentences',
    'no_sentences_writer',
    'no_sentences_llm'
]
x = df_delta.loc[:, feature_cols].values
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
importance = pca.components_[0]
feature_importance = pd.DataFrame(list(zip(feature_cols, importance)), columns=['Feature', 'Importance'])
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

                                  Feature  Importance
2          delta_average_sentence_lengths    0.446753
1   delta Average Dependency tree heights    0.377018
6          delta_avg_right_subtree_height    0.336561
5           delta_avg_left_subtree_height    0.314951
4          delta_average_right_arc_length    0.161124
11                       no_sentences_llm    0.111670
8                delta_num_right_subtrees    0.101071
3           delta_average_left_arc_length    0.081216
7                 delta_num_left_subtrees    0.062296
0              delta average word lengths   -0.100490
9                      delta_no_sentences   -0.339387
10                    no_sentences_writer   -0.511270


Experiment to use delta values for assigning weights(Problem: both LLM and writer summaries will have the same naturalness score)

In [ ]:
feature_cols = [
    'delta average word lengths', 
    'delta Average Dependency tree heights',
    'delta_average_sentence_lengths',
    'delta_average_left_arc_length',
    'delta_average_right_arc_length',
    'delta_avg_left_subtree_height',
    'delta_avg_right_subtree_height',
    'delta_num_left_subtrees',
    'delta_num_right_subtrees',
    'delta_no_sentences',
    'no_sentences_writer',
    'no_sentences_llm'
]
df['overall_writer_better'] = [str(value) for value in df['overall_writer_better']]
df['informative_writer_better'] = [str(value) for value in df['informative_writer_better']]
encoder = LabelEncoder()
df['overall_writer_better'] = encoder.fit_transform(df['overall_writer_better'])
df['informative_writer_better'] = encoder.fit_transform(df['informative_writer_better'])

X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df['overall_writer_better'], test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

importances = clf.feature_importances_
for feature, importance in zip(feature_cols, importances):
    print(f"{feature}: {importance}")


delta average word lengths: 0.16009739275905313
delta_average_sentence_lengths: 0.14983639765647247
delta_average_left_arc_length: 0.13168315442524714
delta_average_right_arc_length: 0.1377145357421385
delta_num_left_subtrees: 0.22254152876206512
delta_num_right_subtrees: 0.19812699065502357
